In [1]:
# 1. Install packages
!pip install evtx pandas scikit-learn scipy seaborn matplotlib pyyaml

# 2. MITRE Threats Table 
import pandas as pd
threats = pd.DataFrame({
    'MITRE_ID': ['T1218.001', 'T1059.001'],
    'Technique': ['rundll32 LOLBin', 'PowerShell Encoded'],
    'EV TX_File': ['1052_cmd.evtx', '1102_powershell.evtx'],
    'Sigma_Target': ['rundll32.*http', 'powershell -enc']
})
threats.to_markdown()  


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 8.8 MB/s eta 0:00:00


'|    | MITRE_ID   | Technique          | EV TX_File           | Sigma_Target    |\n|---:|:-----------|:-------------------|:---------------------|:----------------|\n|  0 | T1218.001  | rundll32 LOLBin    | 1052_cmd.evtx        | rundll32.*http  |\n|  1 | T1059.001  | PowerShell Encoded | 1102_powershell.evtx | powershell -enc |'